In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [3]:
iris = load_iris()
X = iris.data
y = iris.target

In [6]:
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [7]:
y[:5]

array([0, 0, 0, 0, 0])

In [10]:
np.unique(y)

array([0, 1, 2])

In [11]:
X.shape

(150, 4)

In [12]:
y.shape

(150,)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [14]:
X_train_tensor = torch.FloatTensor(X_train)
X_test_tensor = torch.FloatTensor(X_test)

y_train_tensor = torch.LongTensor(y_train)
y_test_tensor = torch.LongTensor(y_test)

In [16]:
X_train_tensor

tensor([[6.1000, 3.0000, 4.6000, 1.4000],
        [7.7000, 3.0000, 6.1000, 2.3000],
        [5.6000, 2.5000, 3.9000, 1.1000],
        [6.4000, 2.8000, 5.6000, 2.1000],
        [5.8000, 2.8000, 5.1000, 2.4000],
        [5.3000, 3.7000, 1.5000, 0.2000],
        [5.5000, 2.3000, 4.0000, 1.3000],
        [5.2000, 3.4000, 1.4000, 0.2000],
        [6.5000, 2.8000, 4.6000, 1.5000],
        [6.7000, 2.5000, 5.8000, 1.8000],
        [6.8000, 3.0000, 5.5000, 2.1000],
        [5.1000, 3.5000, 1.4000, 0.3000],
        [6.0000, 2.2000, 5.0000, 1.5000],
        [6.3000, 2.9000, 5.6000, 1.8000],
        [6.6000, 2.9000, 4.6000, 1.3000],
        [7.7000, 2.6000, 6.9000, 2.3000],
        [5.7000, 3.8000, 1.7000, 0.3000],
        [5.0000, 3.6000, 1.4000, 0.2000],
        [4.8000, 3.0000, 1.4000, 0.3000],
        [5.2000, 2.7000, 3.9000, 1.4000],
        [5.1000, 3.4000, 1.5000, 0.2000],
        [5.5000, 3.5000, 1.3000, 0.2000],
        [7.7000, 3.8000, 6.7000, 2.2000],
        [6.9000, 3.1000, 5.4000, 2

In [17]:
X_train[0], X_train_tensor[0]

(array([6.1, 3. , 4.6, 1.4]), tensor([6.1000, 3.0000, 4.6000, 1.4000]))

In [19]:
type(X_train), type(X_test), type(X_train_tensor), type(X_test_tensor)

(numpy.ndarray, numpy.ndarray, torch.Tensor, torch.Tensor)

In [20]:
batch_size = 5

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=batch_size,
                              shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=batch_size,
                             shuffle=False)

In [21]:
train_dataset, test_dataset

(<torch.utils.data.dataset.TensorDataset at 0x7bad1599cbe0>,
 <torch.utils.data.dataset.TensorDataset at 0x7bad1599db70>)

In [22]:
train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7bad1599f700>,
 <torch.utils.data.dataloader.DataLoader at 0x7bad1599e5f0>)

In [23]:
class NeuralNetwork(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNetwork, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    return self.fc2(self.relu(self.fc1(x)))

In [31]:
# Hyperparameters
input_size = 4
hidden_size = 8
num_classes = 3
learning_rate = 1e-3
num_epochs = 1000

# Create the neural network
model = NeuralNetwork(input_size=input_size,
                      hidden_size=hidden_size,
                      num_classes=num_classes)
model

NeuralNetwork(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=8, out_features=3, bias=True)
)

In [32]:
# Loss function
loss_fn = nn.CrossEntropyLoss()
loss_fn

CrossEntropyLoss()

In [33]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(),
                             lr=learning_rate)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [34]:
# Training the model
for epoch in range(num_epochs):
  for X, y in train_dataloader:
    y_preds = model(X)
    loss = loss_fn(y_preds, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if epoch % 100 == 0:
    print(f'Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}')

Epoch: 1/1000, Loss: 0.8712
Epoch: 101/1000, Loss: 0.0475
Epoch: 201/1000, Loss: 0.0143
Epoch: 301/1000, Loss: 0.0183
Epoch: 401/1000, Loss: 0.0662
Epoch: 501/1000, Loss: 0.1224
Epoch: 601/1000, Loss: 0.4856
Epoch: 701/1000, Loss: 0.0146
Epoch: 801/1000, Loss: 0.0015
Epoch: 901/1000, Loss: 0.5034


In [35]:
model.eval()
with torch.inference_mode():
  correct = 0
  total = 0
  for X, y in test_dataloader:
    y_preds = model(X)
    _, predicted = torch.max(y_preds, 1)
    total += y.size(0)
    correct += (y == predicted).sum().item()

  accuracy = correct / total
  print(f'Test Accuracy: {accuracy:.2f}')

Test Accuracy: 1.00
